In [3]:
# Importar arquivo csv e transformar em dataframe
import pandas as pd

# Ler o arquivo CSV
df = pd.read_csv('./paranavai.csv')
df.columns = ['ibge', 'cd_ui', 'logradouro', 'nr_ui']

# Mostrar as primeiras linhas do DataFrame
print(df.head())


      ibge           cd_ui                    logradouro nr_ui
0  4118402  01.03.020.0095           136 - SALGADO FILHO   950
1  4118402  02.17.011.0151        875 - HELIO DE FREITAS  1293
2  4118402  01.03.369.0078           593 - INGO KRAMBECK   S/N
3  4118402  01.03.369.0015           593 - INGO KRAMBECK   S/N
4  4118402  01.06.275.0060  744 - NELSON MOREIRA BATISTA   S/N


In [4]:
# Cria a coluna quadra 
df['id_quadra'] = df['cd_ui'].str[:9]

# limpa a coluna logradouro 
df[['cd_logradouro', 'nm_logradouro']] = df['logradouro'].str.split(' - ', n=1, expand=True)

df['id_quadra_logradouro'] = df['id_quadra'] + '_' + df['nm_logradouro']
print(df.head())

      ibge           cd_ui                    logradouro nr_ui  id_quadra  \
0  4118402  01.03.020.0095           136 - SALGADO FILHO   950  01.03.020   
1  4118402  02.17.011.0151        875 - HELIO DE FREITAS  1293  02.17.011   
2  4118402  01.03.369.0078           593 - INGO KRAMBECK   S/N  01.03.369   
3  4118402  01.03.369.0015           593 - INGO KRAMBECK   S/N  01.03.369   
4  4118402  01.06.275.0060  744 - NELSON MOREIRA BATISTA   S/N  01.06.275   

  cd_logradouro           nm_logradouro              id_quadra_logradouro  
0           136           SALGADO FILHO           01.03.020_SALGADO FILHO  
1           875        HELIO DE FREITAS        02.17.011_HELIO DE FREITAS  
2           593           INGO KRAMBECK           01.03.369_INGO KRAMBECK  
3           593           INGO KRAMBECK           01.03.369_INGO KRAMBECK  
4           744  NELSON MOREIRA BATISTA  01.06.275_NELSON MOREIRA BATISTA  


In [5]:
## Criar um dataframe apenas com os atributos necessários

# Eliminando linhas com dados da quadra + logradouro repetidos
df_quadra_logradouro = df.drop_duplicates(subset=['id_quadra_logradouro'])
df_ch_quadra_logradouro = df_quadra_logradouro[['id_quadra_logradouro', 'id_quadra', 'cd_logradouro', 'nm_logradouro']]
df_ch_quadra_logradouro.sort_values(by='id_quadra_logradouro', inplace=True)
df_ch_quadra_logradouro.to_csv('./paranavai_logradouros.csv')

print(df_ch_quadra_logradouro.head())

              id_quadra_logradouro  id_quadra cd_logradouro      nm_logradouro
23571           01.01.001_AMAZONAS  01.01.001           105           AMAZONAS
5999            01.01.001_SALVADOR  01.01.001            97           SALVADOR
6         01.01.001_SANTA CATARINA  01.01.001            84     SANTA CATARINA
2790      01.01.002_ETTORE GIOVINE  01.01.002           106     ETTORE GIOVINE
2141   01.01.002_RIO GRANDE DO SUL  01.01.002            83  RIO GRANDE DO SUL


C:\Users\User\AppData\Local\Temp\ipykernel_13884\3525111040.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ch_quadra_logradouro.sort_values(by='id_quadra_logradouro', inplace=True)


In [8]:
# Agrupa as linhas com mesmo id_quadra
logradouros_por_quadra =df_ch_quadra_logradouro.groupby('id_quadra').agg(lambda x: set(x))
logradouros_por_quadra['qt_logradouros'] = df_ch_quadra_logradouro.groupby('id_quadra')[['nm_logradouro']].count()
logradouros_por_quadra.sort_values(by='qt_logradouros', inplace=True, ascending=False)
logradouros_por_quadra.to_csv('./paranavai_logradouros_por_quadra.csv')
#print(logradouros_por_quadra)

# Agrupa as linhas com mesmo nm_logradouro
#quadras_por_logradouro = df_ch_quadra_logradouro.groupby('nm_logradouro')[['id_quadra']].agg(lambda x: set(x))
quadras_por_logradouro = df_ch_quadra_logradouro.groupby('nm_logradouro')[['id_quadra']].agg(lambda x: list(x))

quadras_por_logradouro['qt_quadras'] = df_ch_quadra_logradouro.groupby('nm_logradouro')[['id_quadra']].count()
quadras_por_logradouro.sort_values(by='qt_quadras', inplace=True, ascending=False)
quadras_por_logradouro.to_csv('./paranavai_quadras_por_logradouro.csv')
print(quadras_por_logradouro)

                                                                                          id_quadra  \
nm_logradouro                                                                                         
DEPUTADO HEITOR ALENCAR FURTADO                   [01.02.153, 01.02.155, 01.02.156, 01.02.166, 0...   
PRES TANCREDO NEVES                               [01.02.034, 01.02.035, 01.02.063, 01.02.064, 0...   
AMAZONAS                                          [01.01.001, 01.01.024, 01.01.025, 01.01.048, 0...   
LUIZ SPIGOLON                                     [01.01.012, 01.01.013, 01.01.036, 01.01.037, 0...   
ANTONIO FELIPPE                                   [01.01.061, 01.01.062, 01.01.063, 01.01.065, 0...   
...                                                                                             ...   
JACARANDA - COND RES GREEN PARK                                                         [01.06.124]   
JACARANDA-COND RES BOUGAINVILLE RESIDENCE&RESORT                         

In [89]:
semente = df.loc[df['nm_logradouro'] == 'DEPUTADO HEITOR ALENCAR FURTADO']
print(semente)

          ibge           cd_ui                              logradouro nr_ui  \
7      4118402  01.02.267.0352  6713 - DEPUTADO HEITOR ALENCAR FURTADO  8279   
8      4118402  01.02.267.0352  6713 - DEPUTADO HEITOR ALENCAR FURTADO  8279   
34     4118402  01.04.233.0160  6713 - DEPUTADO HEITOR ALENCAR FURTADO  1038   
35     4118402  01.04.233.0160  6713 - DEPUTADO HEITOR ALENCAR FURTADO  1938   
406    4118402  01.03.040.0109  6713 - DEPUTADO HEITOR ALENCAR FURTADO  4413   
...        ...             ...                                     ...   ...   
51007  4118402  01.04.228.0226  6713 - DEPUTADO HEITOR ALENCAR FURTADO  2415   
51010  4118402  01.04.110.0092  6713 - DEPUTADO HEITOR ALENCAR FURTADO  1399   
51042  4118402  01.04.079.0228  6713 - DEPUTADO HEITOR ALENCAR FURTADO   S/N   
51091  4118402  01.03.063.0379  6713 - DEPUTADO HEITOR ALENCAR FURTADO    67   
51705  4118402  01.03.065.0544  6713 - DEPUTADO HEITOR ALENCAR FURTADO  3260   

       id_quadra cd_logradouro         

In [91]:
paralelas = pd.merge(semente, df, how = 'inner', on = 'id_quadra')
paralelas = paralelas.loc[paralelas['nm_logradouro_y']!='DEPUTADO HEITOR ALENCAR FURTADO']
paralelas = paralelas[['nm_logradouro_y', 'id_quadra']]

paralelas = paralelas.groupby(['nm_logradouro_y']).count()
paralelas.sort_values(by='id_quadra', inplace=True, ascending=False)

print(paralelas)



                        id_quadra
nm_logradouro_y                  
PROF GERALDO LONGO            667
ANTONIO JOSE DA SILVA         627
BARAO DO RIO BRANCO           603
VISCONDE DE GUARAPUAVA        494
PARAGUACU                     420
...                           ...
ANTENOR GRIGOLI                 6
SAO CRISTOVAO                   5
PARANA                          3
ANTONIO VENDRAMIN               3
SAO SALVADOR                    2

[94 rows x 1 columns]
